In [1]:
import sqlite3
import pandas as pd
import datetime as dt
import sqlalchemy.types as sqltypes
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Connect to db
connection = sqlite3.connect('analytics_track.db')
cursor = connection.cursor()

In [28]:
# connection.close()

In [4]:
!ls

Basketball Analytics Prompt.pdf
Clean data & store in DB.ipynb
data.db
NBA Hackathon - Event Codes.txt
NBA Hackathon - Game Lineup Data Sample (50 Games).txt
NBA Hackathon - Play by Play Data Sample (50 Games).txt


# SQL Interpreter for this Data:

data.db file should be in the directory.

Tables:
* event_codes (Event_Msg_Type, Action_Type, Event_Msg_Type_Description, Action_Type_Description)
* game_lineup (Game_id,Period, Person_id, Team_id, status)
* play_by_play (Game_id, Event_Num, Event_Msg_Type, Period, WC_Time, PC_Time, Action_Type, Option1, Option2, Option3, Team_id, Person1, Person2,Team_id_type)

In [3]:
def run_query(qry):
    return pd.read_sql_query(qry, connection)

In [13]:
## EDIT Query Below to interact with DB:

query = """
SELECT DISTINCT Event_Msg_Type, Event_Msg_Type_Description
FROM event_codes
"""

run_query(query)

,Event_Msg_Type,Event_Msg_Type_Description
0,1,Made Shot
1,2,Missed Shot
2,3,Free Throw
3,4,Rebound
4,5,Turnover
5,6,Foul
6,7,Violation
7,8,Substitution
8,9,Timeout
9,10,Jump Ball


## Read in Text Data

### Event Codes 

In [62]:
# Read in Text File
df = pd.read_csv('NBA Hackathon - Event Codes.txt', delimiter='\t')

# Take care of strange NaN value on row index 146:
df.set_value(146, 'Action_Type_Description', 'No Shot')

# Remove extra spaces in last two columns:
df["Event_Msg_Type_Description"] = df["Event_Msg_Type_Description"].map(str.strip)
df["Action_Type_Description"] = df["Action_Type_Description"].map(str.strip)

# Save to DB file in 'event_codes' table:
df.to_sql('event_codes', connection, if_exists='replace') # name of SQL table, connection, append

In [12]:
run_query('SELECT * FROM event_codes').drop(['index'], axis=1).sort_values(by=['Event_Msg_Type', 'Action_Type'])

,Event_Msg_Type,Action_Type,Event_Msg_Type_Description,Action_Type_Description
0,1,0,Made Shot,No Shot
1,1,1,Made Shot,Jump Shot
214,1,2,Made Shot,Running Jump Shot
2,1,3,Made Shot,Hook Shot
3,1,4,Made Shot,Tip Shot
4,1,5,Made Shot,Layup Shot
5,1,6,Made Shot,Driving Layup Shot
6,1,7,Made Shot,Dunk Shot
7,1,8,Made Shot,Slam Dunk Shot
8,1,9,Made Shot,Driving Dunk Shot


In [11]:
pd.options.display.max_rows = 300

In [7]:
# Count how many action types for each Event_Msg
pd.read_sql_query('SELECT Event_Msg_Type, Event_Msg_Type_Description, count(Action_Type) FROM event_codes GROUP BY Event_Msg_Type', connection)
# 13 types of events.

,Event_Msg_Type,Event_Msg_Type_Description,count(Action_Type)
0,1,Made Shot,78
1,2,Missed Shot,78
2,3,Free Throw,19
3,4,Rebound,3
4,5,Turnover,40
5,6,Foul,28
6,7,Violation,8
7,8,Substitution,5
8,9,Timeout,7
9,10,Jump Ball,1


Events that might be useful in keeping track of +/-:
* Made shot (2 or 3)
* Free Throw (0 or 1)
* Start/End Period can refresh iteration
* Ejection - player immediately removed from player tracker
* Substitution - Help keep track of active players (check free throw status first)

Events we don't need to consider:
* Missed shots: 0 points
* Rebound
* Turnover
* Foul (unless it leads to free throw, which is it's own event)
* Violation
* Timeout, JumpBall

### Game Lineups

In [85]:
# Read in Text File
df = pd.read_csv('NBA Hackathon - Game Lineup Data Sample (50 Games).txt', delimiter='\t')

# Save to DB file in 'game_lineup' table:
df.to_sql('game_lineup', connection, if_exists='replace') # name of SQL table, connection, append

In [87]:
print('First 5 rows of DB:')
# First five rows:
pd.read_sql_query('SELECT * FROM game_lineup', connection).head()

# pd.read_sql_query('SELECT DISTINCT Game_id FROM game_lineup', connection)   ## 50 games
# pd.read_sql_query('SELECT DISTINCT Person_id FROM game_lineup', connection)   ## 389 Players
# pd.read_sql_query('SELECT DISTINCT Period FROM game_lineup', connection)    ## 1-5 (OT is >4)
# pd.read_sql_query('SELECT DISTINCT status FROM game_lineup', connection)   ## always A

First 5 rows of DB:


,index,Game_id,Period,Person_id,Team_id,status
0,0,021fd159b55773fba8157e2090fe0fe2,1,881f83d2dee3f18c7d1751659406144e,012059d397c0b7e5a30a5bb89c0b075e,A
1,1,021fd159b55773fba8157e2090fe0fe2,1,27ea17a8685c4919f157e83fe9cb2d9e,cff694c8186a4bd377de400e4f60fe47,A
2,2,021fd159b55773fba8157e2090fe0fe2,1,57bbd7e30bc694aeee9ee40c583e6811,cff694c8186a4bd377de400e4f60fe47,A
3,3,021fd159b55773fba8157e2090fe0fe2,1,cec898a1d355dbfbad8c760615fde1af,012059d397c0b7e5a30a5bb89c0b075e,A
4,4,021fd159b55773fba8157e2090fe0fe2,1,33963fe856a1523ff46438ba07d1d99f,cff694c8186a4bd377de400e4f60fe47,A


### Create Mappings to Turn Codes into Readable Text

In [8]:
game_ids = run_query('SELECT DISTINCT Game_id FROM game_lineup')

for gid in game_ids['Game_id']:
    print(gid)

021fd159b55773fba8157e2090fe0fe2
03a31e84b194d6c8a2eab5d70ba67acf
06bb1d31c63891e2580ff12e4e6505b4
07e76f7482773e81e2351d1692e9e5bb
0868dee930f69a54541d4ae88b841a37
09d46e3d7a8253b7209100650b5afaeb
13ced855d491384876c6ab807bd1d3db
15d76177caa6022156e83774c2e054d3
1eab6189ad9ab246c197575a8c4eebe5
1f9e3cb05c031986cf8bc7c0a84cc517
2bf4ac0ed9ac1aee8767134d62b34dfe
2faabfa663f4dfb9ed83a1482088e092
3152e9c330ce200cc189ae64ebdf41fc
33f631fec90cc1f08bb16cff5ed52f9b
345f285f7c654bf7a03e940a9bc923c3
3b753670a0d0df2c35c7ce2e0bc94e6c
56ccfaf0adead6f4c7236a01ca0cfbdc
598a55c8bf052d039d0fb4ff1a62f98a
7fad2269ee0d11ae5069ff23ecb25913
87d1574f478f37d13789284b96b4f6cb
88012a99d7fd1c169e2360aa5cdf8bfa
896bcac9b0f35b250c9fb7a6325d8599
8fd2943309c7accc78b41e047e1e9393
9db75b921076b5848cba7ca36436147c
9dce47bc3cbabd451a9bab758ca5467f
a109cef8327feae75c3e4885cb4c6e2d
a32200385b4594cb11f8a19e92c96835
a442eeb8e92d2941addba3996bd7d0a0
a55fe197c4ae19094a2303a950c7e70c
b2abcb6a29ed3d99db98084c96638bff
b2c59cbe72

50 Games, 389 Players, 30 Teams Represented in data set. 'Status' column is always "A" so shouldn't matter.

### Play by Play

In [89]:
# Read in Text File
df = pd.read_csv('NBA Hackathon - Play by Play Data Sample (50 Games).txt', delimiter='\t')

# Save to DB file in 'game_lineup' table:
df.to_sql('play_by_play', connection, if_exists='replace') # name of SQL table, connection, append

In [6]:
# First five rows:
df = pd.read_sql_query('SELECT * FROM play_by_play', connection).head()
df.iloc[0:20]

# pd.read_sql_query('SELECT DISTINCT Game_id FROM game_lineup', connection)   ## 50 games
# pd.read_sql_query('SELECT DISTINCT Person_id FROM game_lineup', connection)   ## 389 Players
# pd.read_sql_query('SELECT DISTINCT Period FROM game_lineup', connection)    ## 1-5 (OT is >4)
# pd.read_sql_query('SELECT DISTINCT status FROM game_lineup', connection)   ## always A

,index,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Team_id_type
0,0,021fd159b55773fba8157e2090fe0fe2,0,12,1,546427,7200,0,0,0,0,1473d70e5646a26de3c52aa1abd85b1f,6bcf6c1f8c373d25fca1579bc4464a91,6bcf6c1f8c373d25fca1579bc4464a91,0
1,1,021fd159b55773fba8157e2090fe0fe2,1,10,1,546495,7200,0,0,0,0,012059d397c0b7e5a30a5bb89c0b075e,89706b99ddd00dc05d37ef5cafc04276,307beab25b1021a548b4a47550bc4b25,2
2,2,021fd159b55773fba8157e2090fe0fe2,2,2,1,546665,7050,1,3,0,0,012059d397c0b7e5a30a5bb89c0b075e,cec898a1d355dbfbad8c760615fde1af,6bcf6c1f8c373d25fca1579bc4464a91,2
3,3,021fd159b55773fba8157e2090fe0fe2,3,4,1,546714,6960,0,0,0,0,012059d397c0b7e5a30a5bb89c0b075e,307beab25b1021a548b4a47550bc4b25,6bcf6c1f8c373d25fca1579bc4464a91,2
4,4,021fd159b55773fba8157e2090fe0fe2,6,6,1,546886,6920,4,0,0,0,cff694c8186a4bd377de400e4f60fe47,c00264c3114d23bac482e9de50fb7d28,89706b99ddd00dc05d37ef5cafc04276,3
